# **Group 2 - Work 1**

**Task :** 
Each group should choose between HLL or Boom filters and produce an iPynb file with python code and markdown, which:

- Briefly explains what the algorithm’s purpose is [15%];
- Discuss a possible (real) application of the algorithm [25%];
- Describe the packages/microservices you are going to use [15%];
- Implement the code simulating this real application [45%].

## **Students :**
- Daniel Filipe Vilhena Nunes (101220)
- Prosper Ablordeppey (106382)
- Richard Adolph Aires Jonker (109560)
- Roshan Poudel (109806)

# Hyper Log Log Algorithm (HLL)


_All information in this section is based on (Flajolet et al., 2007)_

HLL is a probabilistic algorithm belonging to the family of Flajolet-Martin (FM) derived algorithms. The way FM works is by counting the trailing zeros of a hashed bit string and returning a probable count of distinct values in a data stream. Derived from it we find the LogLog algorithm and from this we derive the HLL algorithm.
In essence, we count _leading_ zeros from a part of a hashed bit string, store them in an appropriate register and return a probable count of distinct values. 

The rationale is that if a bit string has length $L$, there are $2^L$ possible distinct values for it. If the first bit is 1, now there are only $2^{L-1}$ possible values – half of the total. So, if we denote by N the number of leading zeros of a bit string, we get that it has $2^{L-(N+1)}$ possible values. This is the basis for the HLL algorithm. Now, if we find a string that has $N’>N$ leading zeros, we assume that there are instead $2^{L-(N’+1)}$ distinct values. In essence, we store the highest count of leading zeros. 

If only one register is used, all we get is the highest count of leading zeros which leads to error. For instance, if the highest count is 8 in a 9-bit string, is it correct to assume that there is only one distinct value? Following the procedure in FM and LogLog, HLL also implements several registers, which reduces the error. As such, we define the HLL structure as being composed of $m$ registers where each will hold the highest count of leading zeros of a bit string assigned to it.

Given the object ($d$) of data stream ($\sigma$), we pass it as the argument of a hash function ($h: D \rightarrow \{0,1\}^L$) which return a bit string of length $L$ ($h(d)$). The first $k$ bits of $h(d)$ indicate the register that will hold the count ($m=2^k$). The remaining bits of $h(d)$ (length is $L-k$) are the one where the number of leading zeros are counted. Namely, the value stored corresponds to the position of the leftmost bit with value $1$. Contrary to the usual programming paradigm, the first bit corresponds to a position of 1 and not of 0. If there are no bits with value zero, the value held is $L-k+1$. The highest count corresponding to a register is always the one to be stored.

Afterwards, when the entire data stream has been processed, the product of the harmonic mean of $2^{N_i}$ ($z$), where $N_i$ is the count associated with the $i$-th register, with $m$ gives a first estimation of the true count of distinct values.

## Bias and Range Corrections

Also introduced in (Flajolet et al., 2007) is the use of two corrections to the result previously obtained, these are determined as _bias correction_ ($\alpha_m$) and _range correction_.

### Bias Correction

The parameter is given by:

$$\alpha_m = \left(m \int_0^\infty \left(\log_2\left(\frac{2+u}{1+u}\right)\right)^m du\right)^{-1}$$

For some commonly used values of $m$:
$\alpha_{16} = 0.673$  
$\alpha_{32} = 0.697$  
$\alpha_{64} = 0.709$  
$\alpha_{m\ge128} = \frac{0.7213}{1+\frac{1.079}{m}}$

Multiplying this parameter by the previous estimation, a raw estimation ($E$) is obtained:
$$E = \alpha_m m z$$

### Range Correction

Performance of the algorithm is divided in three ranges (defined through $E$ and $m$, each with an associated correction which gives the final count estimation ($E^*$).

**Small range: $E \le \frac{5}{2}m$**  
> Let $V$ be the number of registers equal to 0. 
If $V \ne 0 \rightarrow E^* := m\log\left(\frac{m}{V}\right)$, otherwise $E^* := E$

**Intermediate range: $\frac{5}{2}m < E \le \frac{1}{30}2^{32}$**  
> No correction is applied, $E^* := E$

**Large range: $E > \frac{1}{30}2^{32}$**  
> Simply set $E^*:= -2^{32} \log\left(1-\frac{E}{2^{32}}\right)$

# Real world uses of HLL
HyperLogLog is mainly used to  estimate the number of unique values within a very large dataset or stream using little memory and time. 

- Counting unique views on a Reddit post.( [View Counting at Reddit](https://www.redditinc.com/blog/view-counting-at-reddit/) )
- Facebook uses HLL in Presto to speed up distinct queries (for example determine the number of distinct people visiting Facebook in the past week). ( [HLL in Presto](https://engineering.fb.com/2018/12/13/data-infrastructure/hyperloglog/) )


- Other possible uses could be counting unique visitors on a website or counting people at a event (using ticket IDs) or counting total visitors at a airport or counting distinct number of cars passing a junction.

### Counting unique visitors on a website
A possible use for HLL algorithm could be counting unique number of visitors on a website. As same user can visit the same website multiple times and usually exact number of visitors is not needed, HLL would be perfect for this application. We can simply use the IP address of the user to determine unique number of visitors. 

# Implementation of HLL

To implement the HLL algorithm, an object-based approach was taken. A class representing it was defined and its methods provide the desired functionalities - from the actual distinct count to comparing the result with an exact answer.

## Methods Description

Methods included in the `HLLAlg` class are:

`hashString(self, value)`
> Takes as input `value`, which is to say, an object of a data stream. This (if not already) is converted to `string` format and passed to a hash function (`sha1` was chosen). This hashed bit string is returned with a fixed length of 24.

`initialise_registers(self)`
> From the defined error rate, $m$ and $k$ are determined. The variable `hll_struct` is initialized as a dictionary of length $m$, with the each key corresponding to the number of the register and each value is initially set at 0.

`alg(self, stream)`
> Implementation of the HLL Algorithm previously described. Takes as input a data stream and returns a probabilistic count of distinct values in the stream.

`verify(self, stream)`
> Verification of the result. Simply prints the result of the implemented algorithm and the exact number of distinct values. The later being found by converting the stream to a Python **`set`** and pass it to Python's `len()` (length) function.

## The Code

In [1]:
import hashlib
import math
import numpy as np
import random
import statistics as st

In [37]:
# error rate allows us to determine the number of registers (m) to be used
error_rate = 0.01
class HLLAlg:

    # First we need to define the hash function, 
    # converting a string with any length 
    # to a string of binary characters (0, 1) with fixed length (24)
    def hashString(self, value):
        if not isinstance(value, str):
            value = str(value)
        
        hashcode=hashlib.sha1(value.encode('utf-8')).hexdigest()
        bin_code = bin(int(hashcode, 16))[-24:].zfill(24)
    
        return bin_code

    # Here registers are initialized as 0, with m being determined from the desired error rate
    def initialise_registers(self):
        #self.k = 4
        if error_rate is not None:
            self.m = (1.04/error_rate)**2
            self.k = math.ceil(math.log(self.m, 2))
        self.m = 2**self.k
        self.hll_struct = {r: 0 for r in range(self.m)}
    

    # Now for the algorithm itself
    def alg(self, stream):
        self.initialise_registers()
        for d in stream:
            # Map the object d from a data stream using the defined hash function
            dh = self.hashString(d)
            # Selecting the first k digits of dh to determine the register (converting the value to int)
            r = int(dh[:self.k], 2)
            # The remaining digits are where we count the leading 0s
            q = dh[self.k:]
            
            # If the hashed string is 000...0, we register its length+1, 
            # as if the first 1 was just outside the string
            if q.find("1") == -1:
                count = len(q)+1
            # Otherwise we find the position of the 1 in the string and just add 1 to it.
            # Equivalent to counting from 1 instead of from 0
            else:
                count = q.find("1")+1
                
            # Now, if the count is higher than what is already in the designated register
            # we keep the new count, otherwise we won't change it
            if (count > self.hll_struct[r]):
                self.hll_struct[r] = count

        # We take all registered counts and use them to make a list of powers of 2
        # Imagine the register has 0, that means there were no leading zeros in a string of length L
        # Which means that there are 2^(L-1) possible values for it
        # Or of the 2^L possible values for said string, half of them don't start with a zero
        # This 2^x gives us the inverse of that probability

        vals = [x for x in self.hll_struct.values() if x != 0] # ignoring the registers with no value
        v = [2**x for x in vals]        
        print("%d/%d registers holding some count" % (len(v), self.m))

                
        # According to the original article, we now take the harmonic mean of the entire list of powers of 2
        z = st.harmonic_mean(v)
        
        # Alfa is a correction parameter also introduced in the article    
        alfa_dict = {16: 0.673, 32: 0.697, 64: 0.709, 128: 0.7213/(1+(1.079/self.m))}
        alfa = alfa_dict[128]

        # So the raw estimation is alfa*m*z. Te article says m^2 instead of m.
        # But in the article z is defined as the harmonic mean/m
        # Which means that this is equivalent
        raw = alfa*self.m*z
        
        # These are the range corrections also introduced in the article
        # The algorithm works really well for that intermediate range. 
        # Below or above that it needs some help
        if raw <= 2.5*self.m:
            print("Small range correction")
            u = len([x for x in self.hll_struct.values() if x==0])
            if u != 0:
                return self.m*math.log(self.m/u)
            else:
                return raw
        elif raw <= (1/30)*2**32:
            print("Intermediate range correction")
            return raw
        else:
            print("Large range correction")
            return -(2**32)*math.log(1-raw/(2**32))
                
        return 0
      

    def verify(self, stream):
        print(f"Estimated distinct values: {self.alg(stream)}")
        print(f"Actual distinct values: {len(set(stream))}")
        print(f"Total length of stream : {len(stream)}")

# Simulating counting Unique IP addresses

In [38]:
ip_stream = []
for i in range(100000):
    
    # Generate random IP address following format XXXX.XXXX.XXXX.XXXX
    ip = ".".join(map(str, (random.randint(0, 255) 
                        for _ in range(4))))
    
    ip_stream.append(ip)

In [39]:
print(ip_stream[:3])

['205.193.86.223', '82.35.139.96', '87.218.135.159']


In [40]:
hll_alg = HLLAlg()
hll_alg.verify(ip_stream)

16350/16384 registers holding some count
Intermediate range correction
Estimated distinct values: 101088.45018584905
Actual distinct values: 99999
Total length of stream : 100000


## References


Flajolet et al. (2007), HyperLogLog: the analysis of a near-optimal cardinality estimation algorithm. 2007 Conference on Analysis of Algorithms, AofA 07, DMTCS proc. AH, 2007, 127–146



In [44]:
x = StreamAlg(ip_stream) 

In [45]:
x.exec()

16350/16384 registers holding some count
Intermediate range correction
101088.45018584905
Estimated distinct values: 101088.45018584905   ≡   101088


In [43]:
import numpy as np
import random 
import math
import hashlib
import statistics


class BaseAlg:
    def __init__(self):
        """
        Initializes the variables for implementation in base class.
        
        Sets:
            self.registers - the struct to hold all counts
            self.k - number of bits of the struct/register to consider
            self.m - actual struct size or cardinality.
            self.error_rate - error rate
            self.results - final estimation
        """
        self.registers = None
        self.k = None
        self.m = None
        self.error_rate = None
        self.results = None
        

    def hashString(self, value_to_hash):
        """
        Hashes the input using the sha1 standard algorithm and returns a 
        string of binary characters (0, 1) with fixed length (24)
        
        Args:
            value_to_hash [str,int] - represents the value to hash.
        Returns:
            padded_binary [str] - a hashed string of binary characters (0, 1)
            with fixed length (24)
            
        Example
            >> self.hashString(25)
            >> '011001000000010000000000'
            >> len(self.hashString(25))
            >> 24

        """

        if not isinstance(value_to_hash, str):
            value_to_hash = str(value_to_hash) # required by the encoding function
        
        hashcode=hashlib.sha1(value_to_hash.encode('utf-8')).hexdigest()
        bin_code = bin(int(hashcode, 16))[-24:].zfill(24) # output padded to 24 bit string
        return bin_code

    
    def initialise_registers(self):
        """
        Creates the required register for algorithm implementation with all
        of the registers initialized as 0. Number of registers are determined
        based on error_rate (if defined) else uses the default m = 2**k.

        The lower the error_rate, the higher the amount of space or register 
        size needed which results on higher the precision to actual count.

        Sets:
            self.m - the actual size of the struct/register
            self.registers - container/struct to hold the register of zeros in each bit position of the struct.
        """
        
        self.k = 4 # used to calculate default m if error not set
        self.error_rate = 0.01 # None, 0.1, 0.01, 0.001 change to see effects

        if self.error_rate is not None:
            self.m = (1.04/self.error_rate)**2
            self.k = math.ceil(math.log(self.m, 2))
        
        self.m = 2**self.k # actual struct size computation
        self.registers = {r: 0 for r in range(self.m)}  # initialize the registers with 0
    

    def alg(self):
        """
        Main register implementation that updates the count of unique elements in the stream.
        ensures for all datastream elements, the register is rightly updated (reason for one-time run)

        Updates:
            self.register - updates the register to for approximation
        """
        
        # ensure initializer is called once 
        if not self.init:
            self.initialise_registers()
            self.init = True

        
        x_hashed = self.hashString(self.x) # obtains the hash of x based on the hashString function
        key = int(x_hashed[:self.k], 2)  # extracts first k bits of x_hashed as key
        q = x_hashed[self.k:] # obtains the last len(x_hashed)-k bitstring of x_hashed 

        
        if q.find('1') == -1:
            value_count = len(q) + 1
        else:
            value_count = q.find('1') + 1
            
            
        if value_count > self.registers[key]:
            self.registers[key] = value_count



    def range_correction_results(self):
        """
        HLL++ range correction implemented in this module
        """
        
        vals = [x for x in self.registers.values() if x != 0]
        v = [2**x for x in vals]

        print("%d/%d registers holding some count" % (len(v), self.m))

        z = statistics.harmonic_mean(v)
        
        alfa_dict = {16: 0.673, 32: 0.697, 64: 0.709, 128: 0.7213/(1+(1.079/self.m))}
        alfa = alfa_dict[128]

        raw = alfa*self.m*z

        if raw <= 2.5*self.m:
            print("Small range correction")
            u = len([x for x in self.registers.values() if x==0])
            if u != 0:
                self.results = self.m*math.log(self.m/u)
            else:
                self.results =  raw
                
        elif raw <= (1/30)*2**32:
            print("Intermediate range correction")
            self.results = raw
            
        else:
            print("Large range correction")
            self.results = -(2**32)*math.log(1-raw/(2**32))
        
        print(self.results)
        return 0


    def verify(self):
        """
        computes the exact results using a deterministic algorithm.
        computation is solely for comparison with the approximated value
        """
        
        print(f"Actual distinct values: {len(set(self.stream))}")
        print(f"Total stream values: {len(self.stream)}")


class StreamAlg(BaseAlg):
    def __init__(self, stream):
        self.stream = stream
        self.init = False
        

    def exec(self):
        """
        Passes each value of the data stream to the alg function for register update. 
        """
        for v in self.stream:
            self.x = v
            self.alg()
        self.range_correction_results()
        print(f"Estimated distinct values: {self.results}   ≡   {int(self.results)}")
